In [1]:
from keras.utils import to_categorical

from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D
import os
import pandas as pd
import numpy as np

In [2]:
TRAIN_DIR='images/train'
TEST_DIR='images/test'

In [3]:
def createdataframe(dir):
    image_paths=[]
    labels=[]
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels    

In [4]:
train=pd.DataFrame()
train['image'],train['label']=createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [6]:
test=pd.DataFrame()
test['image'],test['label']=createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
 print(test)


                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]


In [8]:
from tqdm.notebook import tqdm 

In [9]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        # Load the image in grayscale mode
        img = load_img(image, color_mode='grayscale', target_size=(48, 48))
        img = np.array(img)
        features.append(img)
    # Convert to numpy array and reshape
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

    

In [10]:
train_features=extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [11]:
test_features=extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [12]:
x_train=train_features/255.0
x_test=test_features/255.0


In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
 le=LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [15]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [16]:
y_train=to_categorical(y_train,num_classes = 7)
y_test=to_categorical(y_test,num_classes = 7)

In [17]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

C:\Users\91630\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
model.compile(optimizer ='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [19]:
model.fit(x=x_train,y=y_train,batch_size=128,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 198s 864ms/step - accuracy: 0.2301 - loss: 1.8411 - val_accuracy: 0.2583 - val_loss: 1.8135
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 194s 859ms/step - accuracy: 0.2499 - loss: 1.8137 - val_accuracy: 0.2613 - val_loss: 1.7788
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 197s 870ms/step - accuracy: 0.2674 - loss: 1.7568 - val_accuracy: 0.3433 - val_loss: 1.6407
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 195s 865ms/step - accuracy: 0.3459 - loss: 1.6440 - val_accuracy: 0.4278 - val_loss: 1.4590
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 195s 862ms/step - accuracy: 0.3965 - loss: 1.5461 - val_accuracy: 0.4583 - val_loss: 1.3791
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 194s 857ms/step - accuracy: 0.4398 - loss: 1.4496 - val_accuracy: 0.4994 - val_loss: 1.3158
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 196s 868ms/step - accuracy: 0.4552 - loss: 1.4111 - val_accuracy: 0.5023 - val_loss: 1.2916
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 197s 874ms/step - accuracy: 

In [20]:
model_json = model.to_json()
with open("emotiondectector.json","w") as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")  

In [21]:
from keras.models import model_from_json
 


In [22]:
json_file=open("emotiondetector.json","r")
model_json=json_file.read()
json_file.close()
model=model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [23]:
label=['angry','disgust','fear','happy','neutral','sad','surprise']




In [24]:
def ef(image):
    img = load_img(image, color_mode='grayscale', target_size=(48, 48))
    feature=np.array(img)
    feature=feature.reshape(1,48,48,1)
    return feature/255.0

In [25]:
image='images/train/angry/22.jpg'
print("original image is of angry")
img=ef(image)
pred=model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

original image is of angry
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
model prediction is  angry
